In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [25]:
pip install statsmodel

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement statsmodel (from versions: none)
ERROR: No matching distribution found for statsmodel


In [3]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
from imblearn.combine import SMOTETomek

In [5]:
credit_card_df=pd.read_csv("creditcard.csv")
credit_card_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
credit_card_df.shape

(284807, 31)

In [7]:
#checking the no of missing values in each column
credit_card_df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [8]:
#checking the no of legit and fradulent transactions
credit_card_df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

# Balancing the dataset

In [9]:
legit = credit_card_df[credit_card_df.Class == 0]
fraud = credit_card_df[credit_card_df.Class == 1]

In [10]:
legit.shape , fraud.shape

((284315, 31), (492, 31))

In [11]:
#statistical measure of the legit data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [12]:
#statistical measure of fraud data
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

# Oversampling

In [13]:
x = credit_card_df.drop(['Class'],axis=1)
y = credit_card_df['Class']

In [14]:
x.shape , y.shape

((284807, 30), (284807,))

In [15]:
model = SMOTETomek()
x_new, y_new = model.fit_resample(x,y)

In [17]:
x_new.shape, y_new.shape

((567580, 30), (567580,))

In [18]:
resampled_df = pd.concat([x_new, y_new], axis=1)

In [19]:
resampled_df.shape

(567580, 31)

# Split the data into Training data & Testing Data

In [20]:
X = resampled_df.drop(columns='Class', axis=1)
Y = resampled_df['Class']

In [21]:
X.shape

(567580, 30)

In [22]:
Y.shape

(567580,)

In [34]:
import statsmodels.api as sm
x_cons=sm.add_constant(X)

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_cons,Y,test_size = 0.2 , random_state = 42)

# ModelTraining-Logistic Regression

In [110]:
model=sm.Logit(y_train,X_train)

In [111]:
result_log=model.fit()

         Current function value: 0.081358
         Iterations: 35


C:\Users\ADMIN\anaconda3\envs\MachineLearning\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [112]:
result_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Class   No. Observations:                  787
Model:                          Logit   Df Residuals:                      756
Method:                           MLE   Df Model:                           30
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                  0.8826
Time:                        16:42:29   Log-Likelihood:                -64.029
converged:                      False   LL-Null:                       -545.51
Covariance Type:            nonrobust   LLR p-value:                3.353e-183
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -57.9350   2.67e+05     -0.000      1.000   -5.24e+05    5.24e+05
Time       -2.553e-05    9.2e-06     -2.775      0.006   -4.36e-05    -7.5e-06
V1            10.6688    5.8e+04      0.000      1.000   -1.14e+05    1.14e+05
V2            83.5703   3.94e+05      0.000      1.000   -7.72e+05    7.72e+05
V3           -35.6056   1.52e+05     -0.000      1.000   -2.99e+05    2.99e+05
V4            29.4590   1.24e+05      0.000      1.000   -2.43e+05    2.44e+05
V5             0.5781   1.38e+04    4.2e-05      1.000    -2.7e+04     2.7e+04
V6           -38.6545   1.79e+05     -0.000      1.000   -3.51e+05    3.51e+05
V7          -134.2216   6.17e+05     -0.000      1.000   -1.21e+06    1.21e+06
V8            22.6249   1.05e+05      0.000      1.000   -2.05e+05    2.05e+05
V9           -43.1708   1.88e+05     -0.000      1.000   -3.69e+05    3.69e+05
V10          -97.5023   4.33e+05     -0.000      1.000   -8.48e+05    8.48e+05
V11           79.9781   3.67e+05      0.000      1.000    -7.2e+05     7.2e+05
V12         -144.7212    6.6e+05     -0.000      1.000   -1.29e+06    1.29e+06
V13           -3.7898   1.74e+04     -0.000      1.000   -3.41e+04    3.41e+04
V14         -157.9774    7.2e+05     -0.000      1.000   -1.41e+06    1.41e+06
V15           -5.8231   2.57e+04     -0.000      1.000   -5.04e+04    5.04e+04
V16         -138.6557   6.35e+05     -0.000      1.000   -1.25e+06    1.24e+06
V17         -244.4784   1.11e+06     -0.000      1.000   -2.18e+06    2.18e+06
V18          -92.8596   4.26e+05     -0.000      1.000   -8.35e+05    8.35e+05
V19           39.0570   1.77e+05      0.000      1.000   -3.46e+05    3.47e+05
V20          -27.5938    1.2e+05     -0.000      1.000   -2.35e+05    2.35e+05
V21            6.6628   2.86e+04      0.000      1.000   -5.61e+04    5.61e+04
V22           17.6240   7.83e+04      0.000      1.000   -1.53e+05    1.54e+05
V23           50.4644   2.37e+05      0.000      1.000   -4.64e+05    4.64e+05
V24           -4.2754   2.25e+04     -0.000      1.000   -4.42e+04    4.42e+04
V25           22.1063   1.08e+05      0.000      1.000   -2.11e+05    2.11e+05
V26            5.6848   2.72e+04      0.000      1.000   -5.32e+04    5.32e+04
V27           18.6466   8.46e+04      0.000      1.000   -1.66e+05    1.66e+05
V28           55.3520   2.87e+05      0.000      1.000   -5.63e+05    5.63e+05
Amount         0.6439   2734.812      0.000      1.000   -5359.489    5360.776
==============================================================================

Possibly complete quasi-separation: A fraction 0.57 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [124]:
y_pred = result_log.predict(X_test)
y_pred=cm_df2.round().astype(int)
y_pred

613    1
451    0
731    1
436    0
275    0
      ..
292    0
209    0
506    1
49     0
717    1
Length: 197, dtype: int32

In [125]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.949238578680203
